In [1]:
import pandas as pd
import tweepy
from collections import defaultdict
import praw
import requests
from bs4 import BeautifulSoup

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
consumer_key = 'MivVnKovNAIOFxYgWAWnt2UW9'
consumer_secret = 'hIOLhbIaJ083XgC4kZIYDow0VBS3u5oJ28lK8NNtZcpxGWs3qp'
access_token = '11518572-sHaaBIRgwtITXLOmbXoo3PxeIrxJRgAb0kCF2MJrk'
access_secret = 'Qo0f6A1xermF94mZUKExd1x7xTw2vruEVtj7AOGJ2Y49a'
 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=3, retry_delay=5, retry_errors=set([401, 404, 500, 503]))

In [4]:
def get_profile(user_profile):

    profile = {}

    profile['user_bg_image'] = user_profile.profile_background_image_url
    profile['user_id'] = user_profile.id_str
    profile['user_profile_image'] = user_profile.profile_image_url
    profile['user_created_date'] = user_profile.created_at
    profile['user_followers_count'] = user_profile.followers_count
    profile['user_geo_enabled'] = user_profile.geo_enabled
    profile['user_tweet_count'] = user_profile.statuses_count
    profile['user_location'] = user_profile.location
    profile['user_verified'] = user_profile.verified
    profile['user_time_zone'] = user_profile.time_zone
    profile['user_name'] = user_profile.name
    profile['user_url'] = user_profile.url
    profile['user_screen_name'] = user_profile.screen_name
    profile['user_language'] = user_profile.lang
    profile['user_friends_count'] = user_profile.friends_count
    profile['user_description'] = user_profile.description
    profile['user_listed_count'] = user_profile.listed_count
    profile['userID'] = user_profile.id
    
    return profile

In [5]:
def get_user_profile(user_id):
        
    return get_profile(api.get_user(user_id))

In [6]:
def get_follower_profiles(user_id):
    
    follower_data = defaultdict()
    
    for follower in tweepy.Cursor(api.followers, id=user_id, count=200).items(1000):
        profile = get_profile(follower)
        follower_data[profile['user_screen_name']] = profile
                
    return follower_data

In [ ]:
def get_tweeter_and_retweeter_profiles(tweet_id):
    
    # Get the user IDs for the original author and first 100 retweeters
    profile_ids = [api.get_status(tweet_id).user.id]   
    profile_ids.extend([x.user.id for x in api.retweets(tweet_id)])
    
    data = defaultdict()
    
    # Get their profiles and the profiles of their followers
    for userid in profile_ids:
        print('Total profiles:', len(data))
        
        profile = get_user_profile(userid)
        
        data[profile['user_screen_name']] = profile
                
        data.update(get_follower_profiles(userid))
    
    # Return list of profiles
    return data

In [ ]:
data = get_tweeter_and_retweeter_profiles(tweet_id=778956650602455040)

Total profiles: 0
Total profiles: 118
Total profiles: 512
Total profiles: 1466
Total profiles: 1510
Total profiles: 1854
Total profiles: 1882
Total profiles: 1920
Total profiles: 1973
Total profiles: 1981
Rate limit reached. Sleeping for: 868


## Reddit

In [ ]:
for key in data.keys():
    
    user_agent = "Searching for Twitter user names."
    r = praw.Reddit(user_agent=user_agent)
    
    user = r.get_redditor(key)

    try:
        print(user.created)
    except:
        continue

## Disqus

In [ ]:
for key in data.keys():   
    r = requests.get("https://disqus.com/by/"+key+"/")
    soup = BeautifulSoup(r.text, "lxml")
    if '404' in soup.title.string:
        continue
    else:
        print(key, 'found on Disqus')